In [17]:
"""
network.py
~~~~~~~~~~
A module to implement the stochastic gradient descent learning
algorithm for a feedforward neural network.  Gradients are calculated
using backpropagation.  Note that I have focused on making the code
simple, easily readable, and easily modifiable.  It is not optimized,
and omits many desirable features.
"""

#### Libraries
# Standard library
import random

# Third-party libraries
import numpy as np
import matplotlib.pyplot as plt

In [18]:

# %load mnist_loader.py
"""
mnist_loader
~~~~~~~~~~~~
A library to load the MNIST image data.  For details of the data
structures that are returned, see the doc strings for ``load_data``
and ``load_data_wrapper``.  In practice, ``load_data_wrapper`` is the
function usually called by our neural network code.
"""

#### Libraries
# Standard library
import pickle
import gzip

# Third-party libraries
import numpy as np

def load_data():
    """Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.
    The ``training_data`` is returned as a tuple with two entries.
    The first entry contains the actual training images.  This is a
    numpy ndarray with 50,000 entries.  Each entry is, in turn, a
    numpy ndarray with 784 values, representing the 28 * 28 = 784
    pixels in a single MNIST image.
    The second entry in the ``training_data`` tuple is a numpy ndarray
    containing 50,000 entries.  Those entries are just the digit
    values (0...9) for the corresponding images contained in the first
    entry of the tuple.
    The ``validation_data`` and ``test_data`` are similar, except
    each contains only 10,000 images.
    This is a nice data format, but for use in neural networks it's
    helpful to modify the format of the ``training_data`` a little.
    That's done in the wrapper function ``load_data_wrapper()``, see
    below.
    """
    f = gzip.open('mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.
    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.
    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.
    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [38]:
 """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        
        """
        a = activation
        b = bias
        w = weight
        sigmoid : see sigmoid function
        
        Role : Loops through the whole network and updates each neurons activation using the sigmoid function
        """
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
    
        """Take the training data and make a list out of it"""
        training_data = list(training_data)
        eta = 0.1
        
        """Check if there is data in the test_data"""
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)
        
        """
        Mini-batches: Each mini-batch contains mini_batch_size elements from the training set.
        
        Splits the training data into mini-bachtes, and for each mini-batches we train the network. 
        
        """
        mini_batches = []
        for j in range(5):
            random.shuffle(training_data)
            for k in range(0, len(training_data), mini_batch_size):
                mini_batches.append(training_data[k:k+mini_batch_size])
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Accuracy {} / {}".format(self.evaluate(test_data),n_test));
            else:
                print("Epoch {} complete".format(j))

        """
        Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
    def update_mini_batch(self, mini_batch, eta):
     
        #Building an empty networks filled with empty 0 
        updated_bias = []
        updated_weight = []
        for b in self.biases:
            updated_bias.append(np.zeros(b.shape))
        for w in self.weights:
            updated_weight.append(np.zeros(w.shape))
             
            
        """
        x: features of the instance
        y: label of the instance
        eta : learning rate
        *Chanche i[da=]
        """
        new_updated_bias = []
        new_updated_weight = []

        #Loops through the samples of the mini-batch, calls backprop on each sample.         
        for x, y in mini_batch:
            
            # returns the gradient of the loss function 
            loss_func_bias, loss_func_weight = self.backprop(x, y)
            
            #Updates the mini-batch bias and mini-batch weight by adding their respective loss function to the
            #current mini-batch's network
            
            updated_bias = [ub+lfb for ub, lfb in zip(updated_bias, loss_func_bias)]
            print(updated_bias)
#             count = 0
#             for ub, lfb in zip(updated_bias, loss_func_bias):
#                 print("Count: " + str(count) + "=" + str(lfb + ub))
#                 count += 1
#                 new_updated_bias.append(ub+lfb)
            
            
#             updated_weight = [uw+lfw for uw, lfw in zip(updated_weight, loss_func_weight)]
            
            for uw, lfw in zip(updated_weight, loss_func_weight):
#                 updated_weight.append(uw+lfw)
                new_updated_bias.append(uw + lfw)
#                 uw = uw + lfw
        
        #Updates each weight with the weights calculated in he minibach:
#       new weight= old_weight - new_weight*learning_rate
#       NOTE: can tweak the factor of correction by dividing the eta by the elements in the mini-batch
        #tmpList
        self.weights = [w-(eta/len(mini_batch))*uw
                        for w, uw in zip(self.weights, updated_weight)]
                        #for old_w, batch_w in zip(self.weights, updated_weight)
                         #   self.weights.append(old_w-(eta/len(mini_batch))*batch_w)
        
        #Updates each weight with the bias calculated in he minibach:
#       new bias= old_weight - new_bias*learning_rate
#       NOTE: can tweak the factor of correction by dividing the eta by the elements in the mini-batch
        self.biases = [b-(eta/len(mini_batch))*ub
                       for b, ub in zip(self.biases, updated_bias)]
        

    def backprop(self, x, y):
        """Return a tuple ``(updated_bias, updated_weight)`` representing the
        gradient for the cost function C_x.  ``updated_bias`` and
        ``updated_weight`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        
        #Building an empty networks filled with empty 0 
        updated_bias = [np.zeros(b.shape) for b in self.biases]
        updated_weight = [np.zeros(w.shape) for w in self.weights]
        
        # feedforward
        activation = x
        
        #list to store all the activations, layer by layer
        activation_list = [x]
        
        #list to store all the z vectors, layer by layer
        z_list = [] 
        
        # z : activation without sigmoid 
        # z_list : List of activation of each layer without sigmoid
        # activation : activation of a layer with sigmoid
        # activation_list : List of activation of each layer with sigmoid
        
        # Run through the network and record the activation with and without sigmoid and save it in  
        # z_list(without) and activation_list(with)
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            z_list.append(z)
            activation = sigmoid(z)
            activation_list.append(activation)
            
            
        """
        Now that we ran through the network and calculated the activation we go backward through each layer 
        (from output to input) and update the bias and weight linked to the activations in order to get more 
        accurate results.
        """  
        #Apply output error formula on the last layer of the activation_list and z_list
        output_error = (activation_list[-1] - y) * sigmoid_prime(z_list[-1])
        
        #******Black box needs to be commented******
        #Backpropagate the error
        updated_bias[-1] = output_error
        updated_weight[-1] = np.dot(output_error, activation_list[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists
        for l in range(2, self.num_layers):
            z = z_list[-l]
            sp = sigmoid_prime(z)
            output_error = np.dot(self.weights[-l+1].transpose(), output_error) * sp
            updated_bias[-l] = output_error
            updated_weight[-l] = np.dot(output_error, activation_list[-l-1].transpose())
        return (updated_bias, updated_weight)
        #******End of Black box*******

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))


In [39]:
training_data, validation_data, test_data = load_data_wrapper()
training_data = list(training_data)

In [40]:
net = Network([784, 30, 10])
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

[array([[-2.83407929e-02],
       [-4.75310772e-11],
       [-3.78520200e-09],
       [-9.40784394e-04],
       [ 4.08932193e-10],
       [ 1.14223143e-08],
       [-2.46910658e-02],
       [-8.89667413e-07],
       [ 4.15779005e-05],
       [-1.80366180e-02],
       [ 2.39284722e-08],
       [ 3.52009354e-08],
       [ 4.27628724e-02],
       [-9.86796286e-06],
       [ 4.39966455e-04],
       [-1.30248148e-16],
       [ 2.40322297e-02],
       [-5.48243801e-03],
       [-2.63396074e-04],
       [ 2.47055487e-07],
       [-1.24721036e-08],
       [ 1.36626852e-03],
       [-4.72810654e-07],
       [-3.15188775e-06],
       [ 7.64448905e-03],
       [ 1.76849494e-10],
       [-4.78698583e-06],
       [-7.71289786e-07],
       [ 2.46575322e-07],
       [ 6.18138712e-06]]), array([[ 0.0993139 ],
       [ 0.00325449],
       [ 0.01615065],
       [ 0.00021585],
       [ 0.04320028],
       [ 0.00066945],
       [ 0.00059059],
       [ 0.00617262],
       [ 0.02774529],
       [-0.11730048

[array([[ 8.85928141e-02],
       [ 2.33233942e-02],
       [ 7.21041224e-03],
       [ 2.08114722e-02],
       [ 8.07939428e-02],
       [ 4.93865227e-04],
       [-2.21657674e-02],
       [ 7.83387599e-04],
       [ 1.63978382e-03],
       [ 4.48934977e-03],
       [-3.56427065e-02],
       [ 1.63156192e-03],
       [-1.73781566e-02],
       [-1.11286998e-02],
       [-2.54394322e-02],
       [ 6.19085645e-05],
       [-5.30140711e-02],
       [-4.86456688e-02],
       [ 6.84037810e-03],
       [ 6.20986143e-05],
       [ 6.38855590e-02],
       [-1.05583201e-02],
       [-1.37741552e-03],
       [ 3.93954646e-02],
       [ 8.29422709e-02],
       [-5.31274457e-02],
       [-4.45523628e-04],
       [-3.78063638e-02],
       [ 1.33758532e-01],
       [-3.18906135e-02]]), array([[0.15628179],
       [0.1875264 ],
       [0.18032733],
       [0.28939588],
       [0.05029886],
       [0.01321726],
       [0.17272127],
       [0.14859861],
       [0.17838257],
       [0.07958226]])]
[arra

       [ 0.03003497]])]
[array([[ 1.58749630e-02],
       [ 1.21658973e-02],
       [-2.37468753e-02],
       [-1.67143704e-02],
       [ 4.83158814e-02],
       [-1.76374607e-02],
       [-2.90878510e-02],
       [ 3.53450103e-03],
       [ 5.18696159e-02],
       [ 4.21878451e-02],
       [-3.72135166e-04],
       [-1.64897375e-04],
       [ 5.04459012e-02],
       [-6.68493820e-05],
       [-4.88293448e-03],
       [ 4.08840974e-03],
       [ 1.14987877e-02],
       [ 4.83062532e-04],
       [ 7.70753310e-03],
       [ 1.83443030e-02],
       [-4.27079746e-02],
       [-3.43127408e-03],
       [-1.01325686e-01],
       [-5.21015784e-02],
       [ 7.58738758e-02],
       [ 1.37452575e-02],
       [-4.59640726e-03],
       [-3.22489152e-02],
       [ 8.20425153e-02],
       [ 4.95562476e-03]]), array([[ 0.04642375],
       [-0.00951257],
       [ 0.13521258],
       [ 0.08121169],
       [ 0.15578864],
       [-0.05928405],
       [ 0.16928959],
       [ 0.12926279],
       [ 0.018262

[array([[ 6.14741343e-03],
       [ 4.62090248e-02],
       [ 2.62006773e-02],
       [ 2.53423310e-02],
       [ 3.58979003e-03],
       [-4.90479020e-04],
       [-3.38568888e-02],
       [ 1.74708068e-04],
       [ 1.16724642e-02],
       [ 1.54668209e-01],
       [-5.39535066e-02],
       [ 4.02465676e-04],
       [-1.17956317e-01],
       [-4.72692042e-03],
       [-1.65127098e-01],
       [ 8.99075357e-02],
       [-5.46079850e-02],
       [-6.07974506e-03],
       [ 2.88604278e-02],
       [ 9.75869455e-03],
       [ 3.47339921e-03],
       [-7.03374482e-06],
       [-1.03458624e-03],
       [-7.12922372e-02],
       [-9.07196937e-04],
       [-7.60155529e-03],
       [-1.85309503e-02],
       [-6.40609186e-02],
       [ 2.79963977e-02],
       [ 2.19359894e-03]]), array([[ 0.39836037],
       [ 0.35458229],
       [ 0.53781131],
       [ 0.50278679],
       [ 0.29369173],
       [ 0.08768989],
       [ 0.30906194],
       [ 0.10549842],
       [-0.37617643],
       [ 0.41879829

[array([[ 5.22173507e-05],
       [ 1.70215344e-09],
       [ 9.21088140e-08],
       [-6.88952676e-03],
       [-5.40962122e-06],
       [ 3.40108443e-02],
       [ 2.95154523e-04],
       [ 4.70153826e-04],
       [ 7.26849282e-02],
       [ 7.46206900e-05],
       [-5.01128147e-09],
       [ 3.62602976e-05],
       [-4.33960893e-02],
       [-4.77483325e-06],
       [-3.60635899e-05],
       [ 3.65698363e-05],
       [-1.94462874e-03],
       [-5.60920635e-02],
       [-2.09854211e-03],
       [ 1.10732741e-02],
       [ 7.65066566e-02],
       [-7.29998971e-02],
       [-1.24020542e-04],
       [-2.70542880e-04],
       [-1.13385777e-02],
       [-8.38267553e-05],
       [ 2.12249149e-02],
       [-1.85203103e-03],
       [ 1.07686196e-05],
       [-4.43965777e-03]]), array([[ 3.05342951e-04],
       [ 1.22566446e-02],
       [ 8.48017521e-03],
       [ 1.45497145e-01],
       [ 1.39902868e-01],
       [ 1.67210224e-06],
       [ 3.94098559e-04],
       [-4.98180772e-04],
       [ 

...

[array([[ 0.04474473],
       [ 0.0046334 ],
       [ 0.00690718],
       [-0.00241978],
       [ 0.02275639],
       [-0.00278802],
       [-0.01854653],
       [ 0.03405441],
       [ 0.00174632],
       [ 0.00284097],
       [-0.0862067 ],
       [-0.00172787],
       [ 0.02608468],
       [-0.01270137],
       [-0.13232228],
       [-0.01834485],
       [-0.09383641],
       [ 0.00357088],
       [ 0.02928661],
       [ 0.00267589],
       [-0.00673582],
       [-0.00135177],
       [-0.04879127],
       [ 0.0011966 ],
       [ 0.01458242],
       [-0.03363936],
       [ 0.0245688 ],
       [-0.15304803],
       [ 0.10829413],
       [-0.00018103]]), array([[0.06900862],
       [0.48091324],
       [0.29644634],
       [0.25497004],
       [0.45593392],
       [0.01222676],
       [0.23663496],
       [0.21680716],
       [0.28882393],
       [0.17317618]])]
[array([[ 0.06411903],
       [ 0.00463342],
       [ 0.00702347],
       [-0.00242727],
       [ 0.02275639],
       [-0.003

[array([[ 1.22994709e-01],
       [-1.75171724e-02],
       [ 6.88745540e-02],
       [-3.32438692e-03],
       [ 8.23680697e-02],
       [-1.28960812e-01],
       [-3.39136011e-02],
       [ 4.03256707e-03],
       [ 6.96142057e-02],
       [ 1.10422368e-01],
       [-1.65467320e-02],
       [-8.37016849e-03],
       [ 1.90188779e-01],
       [-4.24624262e-05],
       [-3.29107092e-02],
       [ 6.95606979e-02],
       [-3.86858390e-01],
       [ 3.41291496e-02],
       [ 1.20550879e-01],
       [ 2.53723694e-02],
       [-3.77639226e-02],
       [-1.79702269e-02],
       [-1.96460922e-01],
       [ 1.61393090e-01],
       [ 4.12021288e-02],
       [-1.22906895e-01],
       [ 4.94959705e-02],
       [-3.54141203e-01],
       [ 8.40936212e-02],
       [ 1.09542095e-03]]), array([[-0.10053056],
       [ 0.6234328 ],
       [ 0.73360599],
       [ 0.34297843],
       [ 0.67009374],
       [ 0.20819626],
       [ 0.29207384],
       [ 0.80936154],
       [ 0.10944095],
       [ 0.21150978

[array([[ 5.33874530e-02],
       [ 1.39705128e-02],
       [ 6.21923834e-02],
       [ 1.52312558e-04],
       [ 2.89271304e-04],
       [-1.66817073e-02],
       [-5.34657488e-02],
       [ 2.62676002e-04],
       [ 2.97180013e-02],
       [ 5.62364483e-02],
       [-3.08567611e-03],
       [-4.37340237e-06],
       [ 7.25166320e-02],
       [ 2.38430995e-05],
       [-7.52295087e-02],
       [ 2.15372253e-04],
       [-8.97092902e-02],
       [ 7.53114912e-03],
       [ 2.40460473e-02],
       [-3.29930217e-03],
       [ 1.22895933e-04],
       [-2.79876539e-02],
       [-4.03593793e-02],
       [ 5.69240185e-02],
       [ 4.19446980e-03],
       [-4.12462551e-02],
       [-4.33741221e-04],
       [-1.22817972e-01],
       [ 6.01667121e-02],
       [-1.75731167e-02]]), array([[ 0.02678398],
       [ 0.32247011],
       [ 0.25959727],
       [-0.00881043],
       [ 0.1280513 ],
       [ 0.03139712],
       [ 0.45030507],
       [ 0.16512979],
       [ 0.12971048],
       [ 0.19058692

[array([[ 2.64767947e-02],
       [ 1.37933786e-02],
       [ 1.80853174e-03],
       [-5.12493555e-02],
       [ 7.15141277e-02],
       [-1.17843389e-02],
       [-4.42746799e-02],
       [-2.79040331e-02],
       [ 9.20272914e-02],
       [-8.88964567e-03],
       [-2.13463617e-02],
       [ 2.47741707e-04],
       [ 1.73108971e-02],
       [-1.24573592e-02],
       [-2.22930453e-02],
       [ 1.55771333e-02],
       [ 1.65961644e-02],
       [ 2.20003797e-02],
       [ 7.15696678e-02],
       [ 4.90279154e-02],
       [ 1.45390143e-03],
       [-1.71437833e-02],
       [-1.74407016e-01],
       [-3.68440193e-02],
       [-1.12034147e-02],
       [ 2.18474920e-02],
       [ 4.70556394e-03],
       [-2.91793753e-01],
       [ 3.16458142e-02],
       [ 3.69514801e-02]]), array([[ 0.36426823],
       [ 0.37271782],
       [ 0.51109374],
       [ 0.04096897],
       [ 0.99112236],
       [ 0.11887101],
       [ 0.08608048],
       [ 0.45344919],
       [ 0.00479539],
       [-0.05619405

KeyboardInterrupt: 